In [1]:
import sys
sys.path.append('/Users/bengikoseoglu/anaconda2/pkgs')
sys.path.append('/Library/Python/2.7/site-packages')
sys.path

['',
 '/Users/bengikoseoglu/anaconda2/lib/python27.zip',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-darwin',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-tk',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-old',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-dynload',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/aeosa',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/IPython/extensions',
 '/Users/bengikoseoglu/.ipython',
 '/Users/bengikoseoglu/anaconda2/pkgs',
 '/Library/Python/2.7/site-packages']

In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from datetime import date, timedelta
import datetime as dt
import xgboost
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV  

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
data=pd.read_csv('integrated_data.csv') #new dataset

In [14]:
train_df=data[data.month !=1] 
test_df=data[data.month ==1 ] 
test_df=test_df.reset_index(drop=True)

### Clustering Technique- Bining

In [4]:
bin_unit=pd.read_csv('discretized_item_by_sales.csv')

In [5]:
bin_unit=bin_unit.drop('Unnamed: 0',axis=1)

In [6]:
bin_unit.shape

(12824, 3)

In [7]:
bin_unit.head(1)

,key,total_sales,category
0,10000 XL ( 158-170 ),1,0


In [8]:
bin_unit.category.unique()

array([0, 1, 2, 3, 4])

### Clustering Technique 7 clusters

In [9]:
cluster7=pd.read_csv('clust_7.csv') #cluster numbers

In [10]:
data_cluster=pd.read_csv('data_clustering_R')

In [11]:
data_cluster['clust_7']=cluster7['x']

In [12]:
data_cluster=data_cluster[['key','clust_7']]

In [15]:
train=train_df[['date','key','units']]

In [16]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [17]:
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)
delta = d2 - d1         # timedelta
pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)
post_day=[] 
delta_2 = d3 - d2
for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})
df_days_post = pd.DataFrame({'date':post_day})

# Cluster 30 Days

In [18]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [19]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']] 
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2017-12-19'].sort_values('date')[['key','units']]
sales_prev13=train.loc[train['date']=='2017-12-18'].sort_values('date')[['key','units']]
sales_prev14=train.loc[train['date']=='2017-12-17'].sort_values('date')[['key','units']]
sales_prev15=train.loc[train['date']=='2017-12-16'].sort_values('date')[['key','units']]
sales_prev16=train.loc[train['date']=='2017-12-15'].sort_values('date')[['key','units']]
sales_prev17=train.loc[train['date']=='2017-12-14'].sort_values('date')[['key','units']]
sales_prev18=train.loc[train['date']=='2017-12-13'].sort_values('date')[['key','units']]
sales_prev19=train.loc[train['date']=='2017-12-12'].sort_values('date')[['key','units']]
sales_prev20=train.loc[train['date']=='2017-12-11'].sort_values('date')[['key','units']]
sales_prev21=train.loc[train['date']=='2017-12-10'].sort_values('date')[['key','units']]
sales_prev22=train.loc[train['date']=='2017-12-09'].sort_values('date')[['key','units']]
sales_prev23=train.loc[train['date']=='2017-12-08'].sort_values('date')[['key','units']]
sales_prev24=train.loc[train['date']=='2017-12-07'].sort_values('date')[['key','units']]
sales_prev25=train.loc[train['date']=='2017-12-06'].sort_values('date')[['key','units']]
sales_prev26=train.loc[train['date']=='2017-12-05'].sort_values('date')[['key','units']]
sales_prev27=train.loc[train['date']=='2017-12-04'].sort_values('date')[['key','units']]
sales_prev28=train.loc[train['date']=='2017-12-03'].sort_values('date')[['key','units']]
sales_prev29=train.loc[train['date']=='2017-12-02'].sort_values('date')[['key','units']]
sales_prev30=train.loc[train['date']=='2017-12-01'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-14'})
data_frame_2=data_frame_2.merge(sales_prev18, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-13'})
data_frame_2=data_frame_2.merge(sales_prev19, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-12'})
data_frame_2=data_frame_2.merge(sales_prev20, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-11'})
data_frame_2=data_frame_2.merge(sales_prev21, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-10'})
data_frame_2=data_frame_2.merge(sales_prev22, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-09'})
data_frame_2=data_frame_2.merge(sales_prev23, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-08'})
data_frame_2=data_frame_2.merge(sales_prev24, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-07'})
data_frame_2=data_frame_2.merge(sales_prev25, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-06'})
data_frame_2=data_frame_2.merge(sales_prev26, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-05'})
data_frame_2=data_frame_2.merge(sales_prev27, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-04'})
data_frame_2=data_frame_2.merge(sales_prev28, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-03'})
data_frame_2=data_frame_2.merge(sales_prev29, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-02'})
data_frame_2=data_frame_2.merge(sales_prev30, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-01'})

In [20]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')

### For Time Clustering

In [21]:
c_train=train.merge(data_cluster,on='key',how='inner')
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [22]:
c_dataframe.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2017-12-09,2017-12-08,2017-12-07,2017-12-06,2017-12-05,2017-12-04,2017-12-03,2017-12-02,2017-12-01,clust_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


### For DataFrame Binning

In [23]:
bin_unit=bin_unit.rename(columns={'category':'clust_7'})

In [24]:
c_train_bin=train.merge(bin_unit,on='key',how='inner')
c_dataframe_bin=data_frame.merge(bin_unit,on='key',how='inner')

In [25]:
c_train_bin.head()

,date,key,units,total_sales,clust_7
0,2017-10-01,19671 39 1/3,0.0,1,0
1,2017-10-02,19671 39 1/3,0.0,1,0
2,2017-10-03,19671 39 1/3,0.0,1,0
3,2017-10-04,19671 39 1/3,0.0,1,0
4,2017-10-05,19671 39 1/3,1.0,1,0


In [38]:
c_train_bin.clust_7.unique()

array([0, 1, 2, 3, 4])

In [26]:
c_dataframe_bin.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2017-12-08,2017-12-07,2017-12-06,2017-12-05,2017-12-04,2017-12-03,2017-12-02,2017-12-01,total_sales,clust_7
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,16,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,3


In [27]:
c_dataframe_bin.shape

(12824, 65)

In [31]:
## Running the clusters
def days_30_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=30, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-30)', 'var1(t-29)', 'var1(t-28)', 'var1(t-27)',
       'var1(t-26)', 'var1(t-25)', 'var1(t-24)', 'var1(t-23)',
       'var1(t-22)', 'var1(t-21)', 'var1(t-20)', 'var1(t-19)',
       'var1(t-18)', 'var1(t-17)', 'var1(t-16)', 'var1(t-15)',
       'var1(t-14)', 'var1(t-13)', 'var1(t-12)', 'var1(t-11)',
       'var1(t-10)', 'var1(t-9)', 'var1(t-8)', 'var1(t-7)', 'var1(t-6)',
       'var1(t-5)', 'var1(t-4)', 'var1(t-3)', 'var1(t-2)', 'var1(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=30)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [32]:
# performance Measurement
import math
def cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,date_ass):
    test_data['pid']=test_data["pid"].astype(int)
    test_data['key']=test_data["pid"].astype(str) +" "+ test_data["size"].astype(str) 
    test_data=test_data[['key','stock','sold_out_date']]
    d1 = date(2018, 1, 1)  
    d2 = date(2018, 1, 31)  
    delta = d2 - d1         
    pre_day=[]
    for i in range(delta.days + 1):
        add=str(d1 + timedelta(days=i))
        pre_day.append(add)

    all_=test_data.key.unique()
    n=len(test_data.key.unique())
    i=0
    k=0
    n1=len(pre_day)
    days=[]
    ids=[]
    while(i<n):
        b=all_[i]
        k=0
        while(k<n1):
            a=pre_day[k]
            ids.append(b)
            days.append(a)
            k=k+1
        i=i+1
    test_days = pd.DataFrame({'date':days,'key':ids})
    test_comb=test_days.merge(test_data,on=['key'],how='outer')
    test_comb=test_comb.fillna(0)
    c_test=test_comb.merge(data_cluster,on='key',how='left')
    c_test['clust_7']=c_test['clust_7'].fillna(8)
    check=c_test[['key','sold_out_date','clust_7']]
    check=check.drop_duplicates()
    check['sol_out_pred']=0
    i=0
    all_=c_test.key.unique()
    sold_out=[]
    n=len(all_)
    while(i<n):
        subset_w=c_test.loc[c_test['key']==all_[i]]
        check_w=check.loc[check['key']==all_[i]]
        stock=subset_w['stock'].values[0]
        clust=subset_w['clust_7'].values[0]
        n1=len(pre_day)
        k=0
        while(k<n1): #day based
            day=pre_day[k]
            that_day_sale=0
            if(clust==1):
                that_day_sale=cluster1_frame[day].values[0]
            if(clust==2):
                that_day_sale=cluster2_frame[day].values[0]  
            if(clust==3):
                that_day_sale=cluster3_frame[day].values[0]
            if(clust==4):
                that_day_sale=cluster4_frame[day].values[0]
            if(clust==5):
                that_day_sale=cluster5_frame[day].values[0]
            if(clust==6):
                that_day_sale=cluster6_frame[day].values[0]
            if(clust==7):
                that_day_sale=cluster7_frame[day].values[0]
            if(clust==8):
                sold_out.append(date_ass)
                break
            stock=stock-that_day_sale
            if((stock==0) | (stock<0) ):
                sold_out.append(day)
                break
            if(k==(n1-1)):
                sold_out.append(date_ass)
            k=k+1
        i=i+1
        
    check['sol_out_pred']=sold_out
    check["date_sold_out_date"] = pd.to_datetime(check["sold_out_date"],format="%Y/%m/%d")
    check["date_sold_out_pred"] = pd.to_datetime(check["sol_out_pred"],format="%Y/%m/%d")
    days_differ=check["date_sold_out_date"]-check["date_sold_out_pred"]
    
    check['days_differ']=days_differ
    check['correct_differ']=abs(check['days_differ'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))

    cluster_size=check.clust_7.value_counts()
    by_group=check.groupby(['clust_7']).sum()
    by_group['clust_size']=cluster_size
    by_group['mean']=by_group['correct_differ']/by_group['clust_size']
    print(by_group)
    
    
    days=pd.DataFrame(days_differ,columns=['days_differ'])
    days['correct_differ']=abs(days['days_differ'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))
    summ=sum(days['correct_differ'])
    return(math.sqrt(summ))

In [29]:
# performance Measurement
import math
def cluster_performance_measurement_bin(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,data_cluster,date_ass):
    test_data['pid']=test_data["pid"].astype(int)
    test_data['key']=test_data["pid"].astype(str) +" "+ test_data["size"].astype(str) 
    test_data=test_data[['key','stock','sold_out_date']]
    d1 = date(2018, 1, 1)  
    d2 = date(2018, 1, 31)  
    delta = d2 - d1         
    pre_day=[]
    for i in range(delta.days + 1):
        add=str(d1 + timedelta(days=i))
        pre_day.append(add)

    all_=test_data.key.unique()
    n=len(test_data.key.unique())
    i=0
    k=0
    n1=len(pre_day)
    days=[]
    ids=[]
    while(i<n):
        b=all_[i]
        k=0
        while(k<n1):
            a=pre_day[k]
            ids.append(b)
            days.append(a)
            k=k+1
        i=i+1
    test_days = pd.DataFrame({'date':days,'key':ids})
    test_comb=test_days.merge(test_data,on=['key'],how='outer')
    test_comb=test_comb.fillna(0)
    c_test=test_comb.merge(data_cluster,on='key',how='left')
    check=c_test[['key','sold_out_date','clust_7']]
    check=check.drop_duplicates()
    check['sol_out_pred']=0
    i=0
    all_=c_test.key.unique()
    sold_out=[]
    n=len(all_)
    while(i<n):
        subset_w=c_test.loc[c_test['key']==all_[i]]
        check_w=check.loc[check['key']==all_[i]]
        stock=subset_w['stock'].values[0]
        clust=subset_w['clust_7'].values[0]
        n1=len(pre_day)
        k=0
        while(k<n1): #day based
            day=pre_day[k]
            that_day_sale=0
            if(clust==0):
                that_day_sale=cluster1_frame[day].values[0]
            if(clust==1):
                that_day_sale=cluster2_frame[day].values[0]  
            if(clust==2):
                that_day_sale=cluster3_frame[day].values[0]
            if(clust==3):
                that_day_sale=cluster4_frame[day].values[0]
            if(clust==4):
                that_day_sale=cluster5_frame[day].values[0]
            if(clust=='nan'):
                sold_out.append(date_ass)
                break
            stock=stock-that_day_sale
            if((stock==0) | (stock<0) ):
                sold_out.append(day)
                break
            if(k==(n1-1)):
                sold_out.append(date_ass)
            k=k+1
        i=i+1
        
    check['sol_out_pred']=sold_out
    check["date_sold_out_date"] = pd.to_datetime(check["sold_out_date"],format="%Y/%m/%d")
    check["date_sold_out_pred"] = pd.to_datetime(check["sol_out_pred"],format="%Y/%m/%d")
    days_differ=check["date_sold_out_date"]-check["date_sold_out_pred"]
    
    check['days_differ']=days_differ
    check['correct_differ']=abs(check['days_differ'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))

    cluster_size=data_cluster.clust_7.value_counts()
    by_group=check.groupby(['clust_7']).sum()
    by_group['clust_size']=cluster_size
    by_group['mean']=by_group['correct_differ']/by_group['clust_size']
    print(by_group)
    
    
    days=pd.DataFrame(days_differ,columns=['days_differ'])
    days['correct_differ']=abs(days['days_differ'].astype(dt.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))
    summ=sum(days['correct_differ'])
    return(math.sqrt(summ))

# Original Clusters (Bining)

In [33]:
regr = linear_model.LinearRegression()
cluster1_frame=days_30_cluster(c_dataframe_bin,c_train_bin,0,regr) #0, 1, 2, 3, 4

(313586, 4)
3446


In [34]:
regr = linear_model.LinearRegression()
cluster2_frame=days_30_cluster(c_dataframe_bin,c_train_bin,1,regr) #0, 1, 2, 3, 4

(384748, 4)
4228


In [35]:
regr = linear_model.LinearRegression()
cluster3_frame=days_30_cluster(c_dataframe_bin,c_train_bin,2,regr) #0, 1, 2, 3, 4

(154791, 4)
1701


In [36]:
regr = linear_model.LinearRegression()
cluster4_frame=days_30_cluster(c_dataframe_bin,c_train_bin,3,regr) #0, 1, 2, 3, 4

(146510, 4)
1610


In [37]:
regr = linear_model.LinearRegression()
cluster5_frame=days_30_cluster(c_dataframe_bin,c_train_bin,4,regr) #0, 1, 2, 3, 4

(167349, 4)
1839


### Performance Measurement

We tested on 0 test dataset and saw that, it resulted worse results than clustering 7. Therefore, we didn't tested on other clusters.

In [40]:
test_data=pd.read_csv('test_0.csv') 
cluster_performance_measurement_bin(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,bin_unit,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
0                 11176        3446  3.243180
1                 20610        4228  4.874645
2                 10152        1701  5.968254
3                 12867        1610  7.991925
4                 10395        1839  5.652529


255.3429066960741

# Windowing Lag(30) Regreesion

So far this our best cluster and model. So here we are, running again.

In [95]:
regr = linear_model.LinearRegression()
cluster1_frame=days_30_cluster(c_dataframe,c_train,1,regr)

(806078, 3)
8858


In [96]:
regr = linear_model.LinearRegression()
cluster2_frame=days_30_cluster(c_dataframe,c_train,2,regr)

(156520, 3)
1720


In [97]:
regr = linear_model.LinearRegression()
cluster3_frame=days_30_cluster(c_dataframe,c_train,3,regr)

(25844, 3)
284


In [98]:
regr = linear_model.LinearRegression()
cluster4_frame=days_30_cluster(c_dataframe,c_train,4,regr)

(1001, 3)
11


In [99]:
regr = linear_model.LinearRegression()
cluster5_frame=days_30_cluster(c_dataframe,c_train,5,regr)

(8463, 3)
93


In [100]:
regr = linear_model.LinearRegression()
cluster6_frame=days_30_cluster(c_dataframe,c_train,6,regr)

(3640, 3)
40


In [101]:
regr = linear_model.LinearRegression()
cluster7_frame=days_30_cluster(c_dataframe,c_train,7,regr)

(364, 3)
4


In [102]:
pre_total = pd.concat([cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame])

In [107]:
pre_total=pre_total.reset_index().drop('index',axis=1)

In [109]:
pre_total=pre_total.transpose()

In [110]:
pre_total.to_csv('Windowing_lag30_best_cluster.csv')

### Performance Measurement

In [49]:
test_data=pd.read_csv('test_0.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               39175        4668  8.392245
2.0               11513        1575  7.309841
3.0                1797         281  6.395018
4.0                  67          11  6.090909
5.0                 571          93  6.139785
6.0                 234          40  5.850000
7.0                   9           4  2.250000
8.0               10322        1470  7.021769


252.36481529722008

In [ ]:
test_data=pd.read_csv('test_1.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

In [ ]:
test_data=pd.read_csv('test_2.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

In [ ]:
test_data=pd.read_csv('test_3.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

In [ ]:
test_data=pd.read_csv('test_4.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

# Windowing Lag(30) Random Forest Regression

In [111]:
from sklearn.ensemble import RandomForestRegressor

In [112]:
clf=RandomForestRegressor(max_features='sqrt', min_samples_split=2, n_estimators=10, min_samples_leaf= 4)
cluster1_frame=days_30_cluster(c_dataframe,c_train,1,clf)#c_dataframe,c_train,6,regr,True

(806078, 3)
8858


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [113]:
clf=RandomForestRegressor(max_features='sqrt', min_samples_split=2, n_estimators=10, min_samples_leaf= 4)
cluster2_frame=days_30_cluster(c_dataframe,c_train,2,clf)#c_dataframe,c_train,6,regr,True

(156520, 3)
1720


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [114]:
clf=RandomForestRegressor(max_features='sqrt', min_samples_split=2, n_estimators=10, min_samples_leaf= 4)
cluster3_frame=days_30_cluster(c_dataframe,c_train,3,clf)#c_dataframe,c_train,6,regr,True

(25844, 3)
284


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [115]:
clf=RandomForestRegressor(max_features='sqrt', min_samples_split=2, n_estimators=10, min_samples_leaf= 4)
cluster4_frame=days_30_cluster(c_dataframe,c_train,4,clf)#c_dataframe,c_train,6,regr,True

(1001, 3)
11


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [116]:
clf=RandomForestRegressor(max_features='sqrt', min_samples_split=2, n_estimators=10, min_samples_leaf= 4)
cluster5_frame=days_30_cluster(c_dataframe,c_train,5,clf)#c_dataframe,c_train,6,regr,True

(8463, 3)
93


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [117]:
clf=RandomForestRegressor(max_features='sqrt', min_samples_split=2, n_estimators=10, min_samples_leaf= 4)
cluster6_frame=days_30_cluster(c_dataframe,c_train,6,clf)#c_dataframe,c_train,6,regr,True

(3640, 3)
40


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [118]:
clf=RandomForestRegressor(max_features='sqrt', min_samples_split=2, n_estimators=10, min_samples_leaf= 4)
cluster7_frame=days_30_cluster(c_dataframe,c_train,7,clf)#c_dataframe,c_train,6,regr,True

(364, 3)
4


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


### Performance Measurement

In [119]:
test_data=pd.read_csv('test_0.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40188        4668  8.609254
2.0               12880        1575  8.177778
3.0                1888         281  6.718861
4.0                  74          11  6.727273
5.0                 555          93  5.967742
6.0                 240          40  6.000000
7.0                   9           4  2.250000
8.0               10322        1470  7.021769


257.2080869646209

In [120]:
test_data=pd.read_csv('test_1.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40568        4668  8.690660
2.0               13053        1575  8.287619
3.0                1817         281  6.466192
4.0                  78          11  7.090909
5.0                 493          93  5.301075
6.0                 212          40  5.300000
7.0                  17           4  4.250000
8.0               10299        1470  7.006122


257.9476691113917

In [121]:
test_data=pd.read_csv('test_2.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40414        4668  8.657669
2.0               13049        1575  8.285079
3.0                1912         281  6.804270
4.0                  73          11  6.636364
5.0                 527          93  5.666667
6.0                 200          40  5.000000
7.0                  18           4  4.500000
8.0               10273        1470  6.988435


257.8100075637096

In [122]:
test_data=pd.read_csv('test_3.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40366        4668  8.647386
2.0               13131        1575  8.337143
3.0                1850         281  6.583630
4.0                  78          11  7.090909
5.0                 486          93  5.225806
6.0                 229          40  5.725000
7.0                  12           4  3.000000
8.0               10214        1470  6.948299


257.615993292342

In [123]:
test_data=pd.read_csv('test_4.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               40557        4668  8.688303
2.0               13177        1575  8.366349
3.0                1781         281  6.338078
4.0                  76          11  6.909091
5.0                 551          93  5.924731
6.0                 214          40  5.350000
7.0                  17           4  4.250000
8.0               10187        1470  6.929932


257.9922479455536

# Windowing Lag(30) ANN

In [68]:
from sklearn.neural_network import MLPRegressor

In [71]:
mlp = MLPRegressor()
cluster1_frame=days_30_cluster(c_dataframe,c_train,1,mlp)

(806078, 3)
8858


In [72]:
mlp = MLPRegressor()
cluster2_frame=days_30_cluster(c_dataframe,c_train,2,mlp)

(156520, 3)
1720


In [73]:
mlp = MLPRegressor()
cluster3_frame=days_30_cluster(c_dataframe,c_train,3,mlp)

(25844, 3)
284


In [74]:
mlp = MLPRegressor()
cluster4_frame=days_30_cluster(c_dataframe,c_train,4,mlp)

(1001, 3)
11


In [253]:
cluster5_frame=days_30_cluster(c_dataframe,5,mlp,True)

(8463, 3)
93
91
8463


In [75]:
mlp = MLPRegressor()
cluster6_frame=days_30_cluster(c_dataframe,c_train,6,mlp)

(3640, 3)
40


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


### Performance Measurement

In [77]:
test_data=pd.read_csv('test_0.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               44095        4668  9.446230
2.0               13576        1575  8.619683
3.0                1810         281  6.441281
4.0                  54          11  4.909091
5.0                 530          93  5.698925
6.0                 281          40  7.025000
7.0                  20           4  5.000000
8.0               10322        1470  7.021769


265.87214972614186

In [78]:
test_data=pd.read_csv('test_1.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               44438        4668  9.519709
2.0               13758        1575  8.735238
3.0                1762         281  6.270463
4.0                  48          11  4.363636
5.0                 465          93  5.000000
6.0                 262          40  6.550000
7.0                  31           4  7.750000
8.0               10299        1470  7.006122


266.5764430702758

In [79]:
test_data=pd.read_csv('test_2.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               44224        4668  9.473865
2.0               13738        1575  8.722540
3.0                1831         281  6.516014
4.0                  51          11  4.636364
5.0                 499          93  5.365591
6.0                 261          40  6.525000
7.0                  32           4  8.000000
8.0               10273        1470  6.988435


266.2874386823382

In [80]:
test_data=pd.read_csv('test_3.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               44301        4668  9.490360
2.0               13794        1575  8.758095
3.0                1783         281  6.345196
4.0                  55          11  5.000000
5.0                 459          93  4.935484
6.0                 283          40  7.075000
7.0                  27           4  6.750000
8.0               10214        1470  6.948299


266.3005820496831

In [81]:
test_data=pd.read_csv('test_4.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               44526        4668  9.538560
2.0               13873        1575  8.808254
3.0                1760         281  6.263345
4.0                  52          11  4.727273
5.0                 529          93  5.688172
6.0                 246          40  6.150000
7.0                  30           4  7.500000
8.0               10187        1470  6.929932


266.8389027109803

# Windowing Lag(30) Gradient Boosting Regression

In [82]:
from sklearn import ensemble
from sklearn.metrics import mean_squared_error, r2_score
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
              'learning_rate': 0.01, 'loss': 'ls'}


In [83]:
clf = ensemble.GradientBoostingRegressor(**params)
cluster1_frame=days_30_cluster(c_dataframe,c_train,1,clf)

(806078, 3)
8858


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [84]:
clf = ensemble.GradientBoostingRegressor(**params)
cluster2_frame=days_30_cluster(c_dataframe,c_train,2,clf)

(156520, 3)
1720


In [85]:
clf = ensemble.GradientBoostingRegressor(**params)
cluster3_frame=days_30_cluster(c_dataframe,c_train,3,clf)

(25844, 3)
284


In [86]:
clf = ensemble.GradientBoostingRegressor(**params)
cluster4_frame=days_30_cluster(c_dataframe,c_train,4,clf)

(1001, 3)
11


In [87]:
clf = ensemble.GradientBoostingRegressor(**params)
cluster5_frame=days_30_cluster(c_dataframe,c_train,5,clf)

(8463, 3)
93


In [88]:
clf = ensemble.GradientBoostingRegressor(**params)
cluster6_frame=days_30_cluster(c_dataframe,c_train,6,clf)

(3640, 3)
40


In [89]:
clf = ensemble.GradientBoostingRegressor(**params)
cluster7_frame=days_30_cluster(c_dataframe,c_train,7,clf)

(364, 3)
4


### Performance Measurement

In [90]:
test_data=pd.read_csv('test_0.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               41325        4668  8.852828
2.0               12524        1575  7.951746
3.0                1879         281  6.686833
4.0                  67          11  6.090909
5.0                 569          93  6.118280
6.0                 238          40  5.950000
7.0                  11           4  2.750000
8.0               10322        1470  7.021769


258.7179931894958

In [91]:
test_data=pd.read_csv('test_1.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               41702        4668  8.933590
2.0               12647        1575  8.029841
3.0                1822         281  6.483986
4.0                  66          11  6.000000
5.0                 496          93  5.333333
6.0                 226          40  5.650000
7.0                  20           4  5.000000
8.0               10299        1470  7.006122


259.3800300717077

In [92]:
test_data=pd.read_csv('test_2.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               41522        4668  8.895030
2.0               12614        1575  8.008889
3.0                1908         281  6.790036
4.0                  66          11  6.000000
5.0                 517          93  5.559140
6.0                 227          40  5.675000
7.0                  20           4  5.000000
8.0               10273        1470  6.988435


259.12738180285

In [93]:
test_data=pd.read_csv('test_3.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               41519        4668  8.894387
2.0               12709        1575  8.069206
3.0                1862         281  6.626335
4.0                  70          11  6.363636
5.0                 484          93  5.204301
6.0                 232          40  5.800000
7.0                  17           4  4.250000
8.0               10214        1470  6.948299


259.05018818754024

In [94]:
test_data=pd.read_csv('test_4.csv') 
cluster_performance_measurement(test_data,cluster1_frame,cluster2_frame,cluster3_frame,cluster4_frame,cluster5_frame,cluster6_frame,cluster7_frame,data_cluster,'2018-01-22')

         correct_differ  clust_size      mean
clust_7                                      
1.0               41714        4668  8.936161
2.0               12746        1575  8.092698
3.0                1824         281  6.491103
4.0                  62          11  5.636364
5.0                 556          93  5.978495
6.0                 241          40  6.025000
7.0                  23           4  5.750000
8.0               10187        1470  6.929932


259.52456531126296